In [1]:
import xarray as xr
import pystac_client
import itertools
import time
import numpy as np
import metview as mv
import requests
from urllib.parse import urlparse
import os
os.environ['PROJ_LIB'] = "/opt/conda/envs/geospatial/share/proj"

xr.set_options(keep_attrs=True)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


<span style="color:#336699; font-size:24px;font-weight: bold;">hands on - WMO - World Meteorological Organization </span>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    Alex de Almeida Fernandes - <a href="mailto:alex.fernandes@inpe.br">alex.fernandes@inpe.br</a><br/>
        Felipe Odorizi de Mello - <a href="mailto:felipe.odorizi@inpe.br">felipe.odorizi@inpe.br</a><br/><br/>
    DIPTC – Weather and Climate Forecast Division
    <br/>
    CGCT – General Coordination of Earth Sciences
    <br/>
    INPE – National Institute for Space Research
    <br/>
    Brazil
    <br/><br/>
    Last Updated: November 11, 2025
</div>


<br/>

<div style="text-align: center;font-size: 18px;">
<b>hands on - WMO - World Meteorological Organization </b></div>


# 🌡️ SAMeT
<hr style="border:1px solid #0077b9;">

**SAMeT (South American Mapping of Temperature)** is a product available for South America through the National Institute for Space Research
(CPTEC / INPE) Weather Forecast and Climate Studies website. This product combines the observed data with the ERA5 reanalysis, making a correction of the temperature field using an estimated lapse rate.

**The Lapse Rate** was estimated for four regions located over South America, and for the four seasons of the year. The estimated values for the Lapse Rate were, in all situations, lower than the standard Lapse Rate (-6.5 °C·km-¹ ). The combination of reanalysis and observations, together with the correction of the Lapse Rate produced more accurate temperature fields than those provided by ERA5, especially in regions with more accentuated topography more details in
https://ftp.cptec.inpe.br/modelos/tempo/SAMeT/Rozante_et_al_2021.pdf .

## 👩🏽‍💻 STAC Client API
<hr style="border:1px solid #0077b9;">

To run the example in this Jupyter Notebook, the package will be installed. [pystac-client](https://pystac-client.readthedocs.io/en/latest/).

In [2]:
service = pystac_client.Client.open('https://data.inpe.br/bdc/stac/v1/')
service

<Client id=INPE>

## 🗂 STAC get_collections()
<hr style="border:1px solid #0077b9;">

The method for retrieving all collections from a STAC service.

In [7]:
for collection in service.get_collections():
    print(f'🌡️\033[94m{collection.id}') if collection.id.find('samet') != -1 else print (f'\033[0m{collection.id}')

sentinel-3-olci-l1-bundle-1
GOES19-L2-CMI-1
CB4A-WPM-L4-DN-1
🌡️samet_hourly-1
mosaic-cbers4a-paraiba-3m-1
LCC_L8_30_16D_STK_Cerrado-1
mosaic-landsat-sp-6m-1
CB2B-CCD-L2-DN-1
mosaic-s2-paraiba-3m-1
LCC_L8_30_16D_STK_MataAtlantica-1
mosaic-s2-yanomami_territory-6m-1
LANDSAT-16D-1
LCC_L8_30_16D_STK_Pantanal-1
LCC_L8_30_1M_STK_Cerrado-1
MODISA-OCSMART-RRS-MONTHLY-1
mosaic-s2-brazil-3m-1
sentinel-1-grd-bundle-1
CBERS4-WFI-16D-2
S2_L2A-1
CB4A-WFI-L4-DN-1
CB4A-WFI-L4-SR-1
CB4A-WPM-L2-DN-1
mosaic-landsat-amazon-3m-1
mosaic-s2-amazon-3m-1
AMZ1-WFI-L4-DN-1
S2-3M-1
GOES-GL-DSWRF-Daily-1
CBERS-WFI-8D-1
S2_L1C_BUNDLE-1
LCC_C4_64_1M_STK_GO_PA-SPC-AC-NA-1
CB2B-HRC-L2-DN-1
mosaic-landsat-brazil-6m-1
CB4A-MUX-L4-DN-1
CB4-PAN5M-L4-DN-1
CB4A-WFI-L2-DN-1
CBERS4-MUX-2M-1
S2-16D-2
AMZ1-WFI-L2-DN-1
AMZ1-WFI-L4-SR-1
mod13q1-6.1
charter-wfi-1
CB4-WFI-L4-SR-1
CB4A-MUX-L2-DN-1
mosaic-s2-cerrado-4m-1
S2_L2A_BUNDLE-1
mosaic-cbers4-brazil-3m-1
CB4-MUX-L4-DN-1
myd11a2-6.1
LCC_C4_64_1M_STK_MT_PA-SPC-AC-NA-1
CB4-WFI-L

## 🔍 STAC Filter - get_collection()
<hr style="border:1px solid #0077b9;">

Is used to retrieve detailed metadata about a specific STAC Collection when you know its unique ID. 

In [8]:
collection = service.get_collection('samet_daily-1')
collection

<CollectionClient id=samet_daily-1>

## 📂 STAC get_items()
<hr style="border:1px solid #0077b9;">

The Collection.get_items() method in the PySTAC library is used to retrieve STAC Items associated with a specific STAC Collection.
This method provides a way to access the individual spatiotemporal resources.

In [23]:
from itertools import islice
item = next(islice(itertools.islice(collection.get_items(), 1), 1))

print("ID:", item.id)
print("Datetime:", item.datetime)
print("Bounding Box:", item.bbox)
print("Assets:")
for asset_key, asset in item.assets.items():
    print(f"  - {asset_key}: {asset.href}")


ID: samet_daily_20251113
Datetime: 2025-11-13 00:00:00+00:00
Bounding Box: [-83.0, -56.0, -33.0, 13.0]
Assets:
  - tmax: https://data.inpe.br/bdc/data/samet_daily/DAILY/TMAX/2025/11/SAMeT_CPTEC_TMAX_20251113.nc
  - tmin: https://data.inpe.br/bdc/data/samet_daily/DAILY/TMIN/2025/11/SAMeT_CPTEC_TMIN_20251113.nc
  - tmean: https://data.inpe.br/bdc/data/samet_daily/DAILY/TMED/2025/11/SAMeT_CPTEC_TMED_20251113.nc
  - thumbnail: https://data.inpe.br/bdc/data/samet_daily/FIG/DAILY/TMAX/2025/11/SAMeT_TMAX_20251113.png


## 📄 STAC item
<hr style="border:1px solid #0077b9;">

A **STAC** (SpatioTemporal Asset Catalog) Item is a JSON file that acts as an atomic unit of metadata, describing a single geospatial asset. Its content *includes a GeoJSON feature with additional fields for time, links to related entities, and most importantly, links to the actual data assets it represents, such as images, lidar, or other files*. This makes it a foundational building block for discovering and cataloging spatiotemporal data. 

**Key components of a STAC Item**

***GeoJSON Feature***: The base of the item is a GeoJSON object, providing the spatial extent and geometry of the asset.
Assets: A STAC Item contains a collection of assets, which are links to the actual data files (e.g., raster images, point clouds, or other data). For example, a single satellite scene's different bands (like red, green, blue) would each be a separate asset within a single STAC Item.

***Metadata***: Additional fields are added to the GeoJSON to describe the asset, such as:
Time: The temporal extent of the data.

***Links***: URLs to related items or collections.

***Thumbnails***: Links to smaller preview images for quick browsing.

***Extensibility***: The STAC specification is built with a minimal core and a flexible extension mechanism, allowing it to support a wide variety of data types beyond just imagery. 

In [24]:
item

<Item id=samet_daily_20251113>

In [25]:
item.assets

{'tmax': <Asset href=https://data.inpe.br/bdc/data/samet_daily/DAILY/TMAX/2025/11/SAMeT_CPTEC_TMAX_20251113.nc>,
 'tmin': <Asset href=https://data.inpe.br/bdc/data/samet_daily/DAILY/TMIN/2025/11/SAMeT_CPTEC_TMIN_20251113.nc>,
 'tmean': <Asset href=https://data.inpe.br/bdc/data/samet_daily/DAILY/TMED/2025/11/SAMeT_CPTEC_TMED_20251113.nc>,
 'thumbnail': <Asset href=https://data.inpe.br/bdc/data/samet_daily/FIG/DAILY/TMAX/2025/11/SAMeT_TMAX_20251113.png>}

In [26]:
url = item.assets['tmax'].href
url

'https://data.inpe.br/bdc/data/samet_daily/DAILY/TMAX/2025/11/SAMeT_CPTEC_TMAX_20251113.nc'

## ⬇️ Retrieving SAMet file from URL.
<hr style="border:1px solid #0077b9;">

In [27]:
tmp_dir = "Dados/tmp/"
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
else:
    for file in os.listdir(tmp_dir):
        path = os.path.join(tmp_dir, file)
        if os.path.isfile(path):
            os.remove(path)
        elif os.path.isdir(path):
            os.rmdir(path)

In [28]:
response = requests.get(url)
filename = os.path.basename(urlparse(url).path)
output_path = os.path.join(tmp_dir, filename)

with open(output_path, 'wb') as f:
    f.write(response.content)

nc = mv.read(output_path)


<div style="display: flex; flex-direction: column; align-items: center;">
  <img src="https://metview.readthedocs.io/en/latest/_images/NETCDFVIS.png" alt="NetCDF Visualiser">
  <span style="color:#000000; font-size:16px; padding: 12px;">netcdf_visualiser - Defines the visualisation of NetCDF data
      allowing to specify which variables in the file should be used, and how to interpret them. The generated object can directly be used in plot().
  <a href="https://metview.readthedocs.io/en/latest/gen_files/icon_functions/netcdf_visualiser.html">(details)</a>
  </span>
</div>
<hr style="border:1px solid #0077b9;">

In [29]:
tmax =  mv.netcdf_visualiser(
    netcdf_plot_type="geo_matrix",
    netcdf_latitude_variable="lat",
    netcdf_longitude_variable="lon",
    netcdf_value_variable="tmax",
    netcdf_data=nc,
)

In [30]:
area = [-60,-120,20,0] # S,W,N,E
AMS = mv.geoview(
    map_area_definition = "corners",
    area                = area,
    coastlines = mv.mcoast(
        map_coastline_land_shade        = "on",
        map_coastline_land_shade_colour = "#eeeeee",
        map_grid_latitude_increment     = 10,
        map_grid_longitude_increment    = 10)
    )

auto_style = mv.mcont(contour_automatic_setting = "ecmwf")

shaded = mv.mcont(
    legend                       = "on",
    contour_level_selection_type = "level_list",
    contour_level_list           = [-5, 0, 5, 10, 12.5, 15, 17.5, 20, 22.5, 25, 27.5, 30, 32.5, 35, 37.5, 40, 42.5, 45],
    contour_label                = "off",
    contour_line_colour_rainbow="ON",
    contour_highlight="OFF",
    contour_shade="ON",
    contour_shade_method="AREA_FILL"
    )

Coastlines_FG_ = mv.mcoast(
    map_boundaries="ON",
    map_boundaries_colour="BLACK",
    map_grid_colour="RGB(0,0,0)",
    map_label_colour="RGB(0,0,0)"
    )

_Legend_ = mv.mlegend(
    legend_text_font_size=0.3,
    legend_entry_border_colour="RGB(0,0,0)"
    )

_Text_Plotting_ = mv.mtext(
    text_font_size=0.6,
    text_lines="South American Mapping of Temperature (SAMeT)"
    )

mv.plot(AMS, tmax, shaded, Coastlines_FG_, _Legend_, _Text_Plotting_)


Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')

<img src="https://docs.xarray.dev/en/stable/_static/Xarray_Logo_RGB_Final.svg" align="right" width="100"/>
<span style="color:#336699; font-size:24px;font-weight: bold;">xarray</span>
<hr style="border:2px solid #0077b9;">
<img alt="_images/metview-all-in-one.png" src="https://docs.xarray.dev/en/v2022.09.0/_images/dataset-diagram.png" style="width: 300px;" />
<p><strong>Xarray </strong> provides data models for working with labeled arrays and datasets. Its toolkit includes a broad set of domain-agnostic functions for advanced analytics and visualization with these data structures. Interoperable with the scientific Python ecosystem including NumPy, Dask, Pandas, and Matplotlib.
</p>
<a  href="https://docs.xarray.dev/en/stable/index.html" title="Xarray">documentation</a>

In [31]:
output_path

'Dados/tmp/SAMeT_CPTEC_TMAX_20251113.nc'

In [32]:
import xarray as xr
ds = xr.open_dataset(output_path)
ds


<xarray.Dataset> Size: 11MB
Dimensions:  (lon: 1001, lat: 1381, time: 1)
Coordinates:
  * lon      (lon) float64 8kB -83.0 -82.95 -82.9 -82.85 ... -33.1 -33.05 -33.0
  * lat      (lat) float64 11kB -56.0 -55.95 -55.9 -55.85 ... 12.9 12.95 13.0
  * time     (time) datetime64[ns] 8B 2025-11-13
Data variables:
    tmax     (time, lat, lon) float32 6MB ...
    nobs     (time, lat, lon) float32 6MB ...
Attributes:
    CDI:          Climate Data Interface version 1.8.2 (http://mpimet.mpg.de/...
    Conventions:  CF-1.6
    CDO:          Climate Data Operators version 1.8.2 (http://mpimet.mpg.de/...

In [34]:
tmax = ds['tmax'].values
print(tmax.shape)
print(f"Max: {ds['tmax'].max().values} Min: {ds['tmax'].min().values} Sum: {ds['tmax'].sum().values}")
print(f"Mean: {ds['tmax'].mean().values} Std: {ds['tmax'].std().values} Median: {ds['tmax'].median().values}")
print(f"Quantiles: {ds['tmax'].quantile([0.25, 0.5, 0.75]).values}")

(1, 1381, 1001)
Max: 38.99505615234375 Min: -7.032398223876953 Sum: 17461014.0
Mean: 27.78314781188965 Std: 5.768941879272461 Median: 29.21843147277832
Quantiles: [25.58996868 29.21843147 31.80670834]
